In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import urllib

In [2]:
FILE_NAME = "test"
df = pd.read_csv("{}.csv".format(FILE_NAME))

In [3]:
df

,region,company_type,company_name,inn,incorporation_year,okved_osn_code,init_equities,init_equity_types,purpose,other_reasons_for_check,check_date,type,kpp,type_nasel_punkt,name_nasel_punkt,index,risk_category
0,adygea,юридическое лицо,"муниципальное предприятие ""жилищно-коммунально...",107013764,2007.0,35.0,2.500000e+05,уставный капитал,проверка соблюдения трудового законодательства...,NaN,2018-07-15,выездная,10701001.0,поселок городского типа,тлюстенхабль,385228.0,NaN
1,adygea,юридическое лицо,"публичное акционерное общество ""майкопский ста...",105005101,1999.0,28.0,NaN,NaN,проверка соблюдения трудового законодательства...,NaN,2018-06-15,выездная,10501001.0,город,майкоп,385000.0,NaN
2,adygea,юридическое лицо,"общество с ограниченной ответственностью ""гиаг...",2312184741,2011.0,10.0,1.000000e+04,уставный капитал,проверка соблюдения трудового законодательства...,NaN,2018-08-15,выездная,10101001.0,станица,гиагинская,385601.0,NaN
3,adygea,юридическое лицо,"общество с ограниченной ответственностью ""форм...",107009260,2005.0,22.0,1.000000e+04,уставный капитал,проверка соблюдения трудового законодательства...,NaN,2018-04-15,выездная,10701001.0,поселок городского типа,яблоновский,385140.0,NaN
4,adygea,юридическое лицо,"муниципальное казенное учреждение ""центр админ...",104011222,2007.0,43.0,NaN,NaN,проверка соблюдения трудового законодательства...,NaN,2018-11-15,выездная,10401001.0,поселок,тульский,385730.0,NaN
5,adygea,юридическое лицо,муниципальное бюджетное общеобразовательное уч...,105031750,2000.0,85.0,NaN,NaN,соблюдение обязательных требований пожарной бе...,ЧАСТЬ 9 СТАТЬИ 9 ФЕДЕРАЛЬНОГО ЗАКОНА от 26.12....,2018-06-15,выездная,10501001.0,город,майкоп,385012.0,NaN
6,adygea,юридическое лицо,муниципальное бюджетное общеобразовательное уч...,105033420,2001.0,85.0,NaN,NaN,соблюдение обязательных требований пожарной бе...,ЧАСТЬ 9 СТАТЬИ 9 ФЕДЕРАЛЬНОГО ЗАКОНА от 26.12....,2018-12-15,выездная,10501001.0,город,майкоп,385064.0,NaN
7,adygea,юридическое лицо,муниципальное бюджетное общеобразовательное уч...,105031920,2000.0,85.0,NaN,NaN,соблюдение обязательных требований пожарной бе...,ЧАСТЬ 9 СТАТЬИ 9 ФЕДЕРАЛЬНОГО ЗАКОНА от 26.12....,2018-12-15,выездная,10501001.0,город,майкоп,385077.0,NaN
8,adygea,юридическое лицо,муниципальное бюджетное общеобразовательное уч...,105032909,2000.0,85.0,NaN,NaN,соблюдение обязательных требований пожарной бе...,ЧАСТЬ 9 СТАТЬИ 9 ФЕДЕРАЛЬНОГО ЗАКОНА от 26.12....,2018-10-15,выездная,10501001.0,город,майкоп,385002.0,NaN
9,adygea,юридическое лицо,государственное казенное общеобразовательное у...,105033490,2000.0,85.0,NaN,NaN,соблюдение обязательных требований пожарной бе...,ЧАСТЬ 9 СТАТЬИ 9 ФЕДЕРАЛЬНОГО ЗАКОНА от 26.12....,2018-04-15,выездная,10501001.0,город,майкоп,385017.0,NaN


In [4]:
def transform_med(row): # ~100
    if str(row.purpose) != 'nan' and ('государственный контроль за обращением медицинских изделий' in row.purpose or 'контроль качества и безопасности медицинской деятельности' in row.purpose):
        row.purpose = 'государственный контроль за обращением медицинских изделий'
    return row

def transform_san_epidem(row): # 94
    if str(row.purpose) == 'nan':
        return row
    if 'санитарно-эпидемиологический надзор' in row.purpose:
        row.purpose = 'санитарно-эпидемиологический надзор'
    elif 'санитарно-эпидемиологического благополучия' in row.purpose:
        if 'медицин' in row.purpose or 'помощ' in row.purpose:
            row.purpose = 'санитарно-эпидемиологического благополучия медицина'
        elif 'школ' in row.purpose or 'образова' in row.purpose:
            row.purpose = 'санитарно-эпидемиологического благополучия образование'
        elif 'лагер' in row.purpose:
            row.purpose = 'санитарно-эпидемиологического благополучия лагерь'
        else:
            row.purpose = 'санитарно-эпидемиологического благополучия'
    return row

def transform_fire(row): #103
    if str(row.purpose) == 'nan':
        return row
    if 'пожар' in row.purpose:
        row.purpose = 'пожар'
    return row

def transform_eco(row): #31
    if str(row.purpose) == 'nan':
        return row
    if 'экол' in row.purpose:
        row.purpose = 'экология'
    return row

df = df.apply(transform_med, axis=1)
df = df.apply(transform_san_epidem, axis=1)
df = df.apply(transform_fire, axis=1)
df = df.apply(transform_eco, axis=1)

In [5]:
df.purpose.unique(), len(df.purpose.unique())

(array([ 'проверка соблюдения трудового законодательства и иных нормативных правовых актов, содержащих нормы трудового права',
        'пожар',
        'соблюдение обязательных требований в области гражданской обороны и защиты от чрезвычайных ситуаций',
        'соблюдение требований законов 7-фз от 10.01.2002г. "об охране окружающей среды", 96-фз от 4.05.1999г. "об охране атмосферного воздуха", 89-фз от 24.06.1998г. "об отходах производства и потребления"',
        'санитарно-эпидемиологического благополучия',
        'санитарно-эпидемиологического благополучия медицина',
        'соблюдение требований действующего земельного законодательства рф',
        'лицензионный контроль осуществления фармацевтической деятельности федеральный государственный надзор в сфере обращения лекарственных средств',
        'государственный контроль за обращением медицинских изделий',
        'соблюдение обязательных требований ветеринарного законодательства',
        'соблюдение обязательных требований 

In [6]:
cnt = 0
for p in df.purpose.unique():
    if str(p) != 'nan' and 'соблюдени' not in p and 'соответстви' not in p:
        print(p)
        print()
        cnt += 1
#     if str(p) != 'nan' and 'государственный контроль за обращением медицинских изделий' in p:
#         if 'государственный надзор в сфере обращения лекарственных средств' not in p:
#             print(p)
#         cnt += 1
print(cnt)

пожар

санитарно-эпидемиологического благополучия

санитарно-эпидемиологического благополучия медицина

лицензионный контроль осуществления фармацевтической деятельности федеральный государственный надзор в сфере обращения лекарственных средств

государственный контроль за обращением медицинских изделий

экология

лицензионный контроль фармацевтической деятельности

лицензионный контроль медицинской деятельности лицензионный контроль осуществления деятельности по наркотических средств, психотропных веществ и их прекурсоров, культивированию наркосодержащих растений лицензионный контроль фармацевтической деятельности

государственный земельный надзор, ст. 71, 71.1 земельного кодекса рф

государственный ветеринарный надзор, закон рф от 14.05.1993 4979-1 "о ветеринарии"

проверка исполнения требований федерального закона от 30.12.2004г. №214-фз

лицензионный контроль осуществления медицинской деятельности

обеспечение лицензионного контроля на основании фз №99-фз от 04.05.11 ст.19 п.9.2

c

In [7]:
cnt = 0
for p in df.purpose.unique():
    if str(p) != 'nan' and 'медицин' in p:
        print(p)
        print()
        cnt += 1
#     if str(p) != 'nan' and 'государственный контроль за обращением медицинских изделий' in p:
#         if 'государственный надзор в сфере обращения лекарственных средств' not in p:
#             print(p)
#         cnt += 1
print(cnt)

санитарно-эпидемиологического благополучия медицина

государственный контроль за обращением медицинских изделий

лицензионный контроль медицинской деятельности лицензионный контроль осуществления деятельности по наркотических средств, психотропных веществ и их прекурсоров, культивированию наркосодержащих растений лицензионный контроль фармацевтической деятельности

соответствие требованиям фз о санитарно-эпидемиологическом благополучии населения 52-фз от 30.03.1999 от 07.02.1992 2300-1-фз о защите прав потребителей", фз от 30.03.1995 г. № 38 о предупреждении распространения в россии заболевания, вызывае-мого вич санпин 2.2.1/2.1.1.1278-03 гигиенические требования к естественному, ис-кусственному и совмещенному освещению жилых и общественных зданий,санпин 2.1.4.1074-01 питьевая вода. гигиенически требования к качеству воды централизован-ных систем питьевого водоснабжения. контроль качества, санпин 2.1.7.2790-10 "сани-тарно-эпидемиологические требования к обращению с медицинскими отходам

In [8]:
df.to_csv("{}.csv".format(FILE_NAME))